In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.utils import img_to_array
import os
from random import shuffle


## Data Augmentation

In [ ]:
# # Define data augmentation parameters (adjust as needed)
# datagen = ImageDataGenerator(
#     rotation_range=20,  
#     shear_range=0.2,     
#     zoom_range=0.2,      
#     horizontal_flip=True, 
#     brightness_range=(0.5, 1.5)  # Adjust brightness range
# )

# # Define path to your image folder
# image_dir = "Train/Tv"

# # Define output directory for augmented images (create if it doesn't exist)
# save_to_dir = "Train/Tv"

# # Loop through each image in the folder
# for filename in os.listdir(image_dir):
#     # Load the image
#     img = load_img(os.path.join(image_dir, filename))

#     # Convert to array
#     x = img_to_array(img)

#     # Reshape (optional, based on your model's requirements)
#     x = x.reshape((1,) + x.shape)  # Reshape if your model expects a specific format

#     # Generate and save augmented images
#     i = 0
#     for batch in datagen.flow(x, batch_size=1, save_to_dir=save_to_dir, save_prefix="aug_", save_format="jpeg"):
#         i += 1
#         # You can adjust the number of augmented images per original image here (e.g., i > 5)
#         if i > 1:
#             break
# print(f"Successfully augmented and saved images to {save_to_dir}")


## Modeling

In [2]:
base_model = tf.keras.applications.MobileNetV2(input_shape=(224,224,3),weights="imagenet",include_top=False)

In [3]:
x = tf.keras.layers.GlobalAvgPool2D()(base_model.output)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(128,activation="relu")(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(64,activation="relu")(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(4,activation="softmax")(x)

model = tf.keras.Model(inputs = base_model.input , outputs = x)


In [4]:
#freezing the original model's layers
for layer in base_model.layers:
    layer.trainable = False

In [5]:
model.compile(optimizer="adam",loss="sparse_categorical_crossentropy",metrics=["accuracy"])

In [6]:
# define the directory containing training images
target_size = (224, 224)

# initialize lists to store images and labels
images = []
labels = []

for filename in os.listdir("Train/Laptop"):
    if filename.endswith('.jpg') or filename.endswith("png") or filename.endswith("jpeg"):
        # load and preprocess the image
        img = tf.keras.preprocessing.image.load_img(os.path.join("Train/Laptop", filename), target_size=target_size)
        img_array = tf.keras.preprocessing.image.img_to_array(img)
        img_array = tf.keras.applications.mobilenet_v2.preprocess_input(img_array)
        
       # add the preprocessed image to the list of images
        images.append(img_array)
        labels.append(0)
for filename in os.listdir("Train/Printer"):
    if filename.endswith('.jpg') or filename.endswith("png") or filename.endswith("jpeg"):
        # load and preprocess the image
        img = tf.keras.preprocessing.image.load_img(os.path.join("Train/Printer", filename), target_size=target_size)
        img_array = tf.keras.preprocessing.image.img_to_array(img)
        img_array = tf.keras.applications.mobilenet_v2.preprocess_input(img_array)
        
       # add the preprocessed image to the list of images
        images.append(img_array)
        labels.append(1)
for filename in os.listdir("Train/Smartphone"):
    if filename.endswith('.jpg') or filename.endswith("png") or filename.endswith("jpeg"):
        # load and preprocess the image
        img = tf.keras.preprocessing.image.load_img(os.path.join("Train/Smartphone", filename), target_size=target_size)
        img_array = tf.keras.preprocessing.image.img_to_array(img)
        img_array = tf.keras.applications.mobilenet_v2.preprocess_input(img_array)
        
       # add the preprocessed image to the list of images
        images.append(img_array)
        labels.append(2)
for filename in os.listdir("Train/Tv"):
    if filename.endswith('.jpg') or filename.endswith("png") or filename.endswith("jpeg"):
        # load and preprocess the image
        img = tf.keras.preprocessing.image.load_img(os.path.join("Train/Tv", filename), target_size=target_size)
        img_array = tf.keras.preprocessing.image.img_to_array(img)
        img_array = tf.keras.applications.mobilenet_v2.preprocess_input(img_array)
        
       # add the preprocessed image to the list of images
        images.append(img_array)
        labels.append(3)
        
# convert lists to numpy arrays
X_train = np.array(images)
Y_train = np.array(labels)

In [7]:
X_train.shape

(7294, 224, 224, 3)

In [8]:
# Combine data and labels into a single list
data_and_labels = list(zip(X_train, Y_train))

# Shuffle the combined list
shuffle(data_and_labels)

# Extract shuffled images and labels back into separate arrays
X_train, Y_train = zip(*data_and_labels)

# Convert back to NumPy arrays if necessary
X_train = np.array(X_train)
Y_train = np.array(Y_train)

In [9]:
model.fit(X_train,Y_train,epochs = 10)

Epoch 1/10
228/228 ━━━━━━━━━━━━━━━━━━━━ 97s 382ms/step - accuracy: 0.7950 - loss: 0.5381
Epoch 2/10
228/228 ━━━━━━━━━━━━━━━━━━━━ 83s 365ms/step - accuracy: 0.9481 - loss: 0.1503
Epoch 3/10
228/228 ━━━━━━━━━━━━━━━━━━━━ 85s 375ms/step - accuracy: 0.9629 - loss: 0.1075
Epoch 4/10
228/228 ━━━━━━━━━━━━━━━━━━━━ 95s 418ms/step - accuracy: 0.9776 - loss: 0.0714
Epoch 5/10
228/228 ━━━━━━━━━━━━━━━━━━━━ 93s 406ms/step - accuracy: 0.9838 - loss: 0.0500
Epoch 6/10
228/228 ━━━━━━━━━━━━━━━━━━━━ 83s 363ms/step - accuracy: 0.9838 - loss: 0.0488
Epoch 7/10
228/228 ━━━━━━━━━━━━━━━━━━━━ 90s 395ms/step - accuracy: 0.9897 - loss: 0.0307
Epoch 8/10
228/228 ━━━━━━━━━━━━━━━━━━━━ 100s 440ms/step - accuracy: 0.9894 - loss: 0.0335
Epoch 9/10
228/228 ━━━━━━━━━━━━━━━━━━━━ 78s 340ms/step - accuracy: 0.9944 - loss: 0.0189
Epoch 10/10
228/228 ━━━━━━━━━━━━━━━━━━━━ 103s 453ms/step - accuracy: 0.9922 - loss: 0.0216


In [10]:
# define the directory containing training images
target_size = (224, 224)

# initialize lists to store images and labels
images = []
labels = []

for filename in os.listdir("Test/Laptop"):
    if filename.endswith('.jpg') or filename.endswith("png") or filename.endswith("jpeg"):
        # load and preprocess the image
        img = tf.keras.preprocessing.image.load_img(os.path.join("Test/Laptop", filename), target_size=target_size)
        img_array = tf.keras.preprocessing.image.img_to_array(img)
        img_array = tf.keras.applications.mobilenet_v2.preprocess_input(img_array)
        
       # add the preprocessed image to the list of images
        images.append(img_array)
        labels.append(0)
for filename in os.listdir("Test/Printer"):
    if filename.endswith('.jpg') or filename.endswith("png") or filename.endswith("jpeg"):
        # load and preprocess the image
        img = tf.keras.preprocessing.image.load_img(os.path.join("Test/Printer", filename), target_size=target_size)
        img_array = tf.keras.preprocessing.image.img_to_array(img)
        img_array = tf.keras.applications.mobilenet_v2.preprocess_input(img_array)
        
       # add the preprocessed image to the list of images
        images.append(img_array)
        labels.append(1)
for filename in os.listdir("Test/Smartphone"):
    if filename.endswith('.jpg') or filename.endswith("png") or filename.endswith("jpeg"):
        # load and preprocess the image
        img = tf.keras.preprocessing.image.load_img(os.path.join("Test/Smartphone", filename), target_size=target_size)
        img_array = tf.keras.preprocessing.image.img_to_array(img)
        img_array = tf.keras.applications.mobilenet_v2.preprocess_input(img_array)
        
       # add the preprocessed image to the list of images
        images.append(img_array)
        labels.append(2)
for filename in os.listdir("Test/Tv"):
    if filename.endswith('.jpg') or filename.endswith("png") or filename.endswith("jpeg"):
        # load and preprocess the image
        img = tf.keras.preprocessing.image.load_img(os.path.join("Test/Tv", filename), target_size=target_size)
        img_array = tf.keras.preprocessing.image.img_to_array(img)
        img_array = tf.keras.applications.mobilenet_v2.preprocess_input(img_array)
        
       # add the preprocessed image to the list of images
        images.append(img_array)
        labels.append(3)
        
# convert lists to numpy arrays
X_test = np.array(images)
Y_test = np.array(labels)

In [11]:
X_test.shape

(505, 224, 224, 3)

In [12]:
predictions_array = model.predict(X_test)

16/16 ━━━━━━━━━━━━━━━━━━━━ 10s 521ms/step


In [13]:
predictions = np.argmax(predictions_array,axis=1)

In [20]:
predictions[42:42+146]

array([1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1,
       1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int64)

In [15]:
#Overall 
accuracy_metric = tf.keras.metrics.Accuracy()
accuracy_metric.update_state(Y_test,predictions)
accuracy = accuracy_metric.result().numpy()
print("Accuracy:", accuracy)

Accuracy: 0.9643564


In [16]:
#Laptop
accuracy_metric = tf.keras.metrics.Accuracy()
accuracy_metric.update_state(Y_test[:42],predictions[:42])
accuracy = accuracy_metric.result().numpy()
print("Accuracy:", accuracy)

Accuracy: 0.97619045


In [17]:
#Printer
accuracy_metric = tf.keras.metrics.Accuracy()
accuracy_metric.update_state(Y_test[42:42+146],predictions[42:42+146])
accuracy = accuracy_metric.result().numpy()
print("Accuracy:", accuracy)

Accuracy: 0.94520545


In [18]:
#Smartphone
accuracy_metric = tf.keras.metrics.Accuracy()
accuracy_metric.update_state(Y_test[42+146:42+146+192],predictions[42+146:42+146+192])
accuracy = accuracy_metric.result().numpy()
print("Accuracy:", accuracy)

Accuracy: 0.9739583


In [19]:
#Tv
accuracy_metric = tf.keras.metrics.Accuracy()
accuracy_metric.update_state(Y_test[42+146+192:],predictions[42+146+192:])
accuracy = accuracy_metric.result().numpy()
print("Accuracy:", accuracy)

Accuracy: 0.968


In [24]:
# model.save("ItModelV1.keras")

In [30]:
# model = tf.keras.models.load_model("ItModelV1.keras")

c:\Users\ghass\anaconda3\envs\yolo\Lib\site-packages\keras\src\saving\saving_lib.py:396: UserWarning: Skipping variable loading for optimizer 'adam', because it has 326 variables whereas the saved optimizer has 14 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))
